<a href="https://colab.research.google.com/github/Suresh045/TNSDC/blob/main/3_DC_Grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing the Libraies
import numpy as np
import pandas as pd

In [3]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [6]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [7]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [8]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)


In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],   # Split quality measure
    'splitter': ['best', 'random'],                 # Node splitting strategy
    'max_depth': [None, 10, 20, 30],                # Tree depth
    'min_samples_split': [2, 10, 20],               # Min samples to split
    'min_samples_leaf': [1, 5, 10]                  # Min samples per leaf
}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10, 20],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [14]:

re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [16]:
# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 5, 'min_samples_split': 2, 'splitter': 'random'}: 0.8882608695652173


In [18]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[53  5]
 [ 4 18]]


In [19]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.91      0.92        58
           1       0.78      0.82      0.80        22

    accuracy                           0.89        80
   macro avg       0.86      0.87      0.86        80
weighted avg       0.89      0.89      0.89        80



In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.9298589341692791)

In [21]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [22]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))

In [23]:
table=pd.DataFrame.from_dict(re)

In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002971,0.000618,0.004999,0.001670,gini,None,1,2,best,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.875759,0.779162,0.860772,0.844872,0.855335,0.843180,0.033525,188
1,0.002368,0.000126,0.004191,0.000519,gini,None,1,2,random,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.860542,0.825502,0.907389,0.876518,0.850259,0.864042,0.027284,154
2,0.002455,0.000075,0.004496,0.000505,gini,None,1,10,best,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.825822,0.776515,0.843750,0.875000,0.835227,0.831263,0.031973,204
3,0.002910,0.001130,0.005858,0.002262,gini,None,1,10,random,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.891533,0.787707,0.891711,0.891049,0.936903,0.879781,0.049291,87
4,0.002425,0.000102,0.009564,0.005843,gini,None,1,20,best,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.842448,0.823187,0.892138,0.891049,0.904221,0.870609,0.031790,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0.003327,0.002561,0.004222,0.000691,log_loss,30,10,2,random,"{'criterion': 'log_loss', 'max_depth': 30, 'mi...",0.645878,0.825502,0.812500,0.858747,0.921526,0.812831,0.091604,215
212,0.004006,0.001982,0.010367,0.000980,log_loss,30,10,10,best,"{'criterion': 'log_loss', 'max_depth': 30, 'mi...",0.842448,0.858747,0.891711,0.906923,0.952916,0.890549,0.038689,9
213,0.005174,0.003205,0.007394,0.002260,log_loss,30,10,10,random,"{'criterion': 'log_loss', 'max_depth': 30, 'mi...",0.830206,0.825502,0.791040,0.784535,0.936903,0.833637,0.054713,201
214,0.004261,0.002126,0.007281,0.001494,log_loss,30,10,20,best,"{'criterion': 'log_loss', 'max_depth': 30, 'mi...",0.842448,0.858747,0.891711,0.875000,0.952916,0.884164,0.038089,39
